In [2]:
from aimet_torch.cross_layer_equalization import equalize_model
from torchvision import models
import torch
import os
import sys
import torch.optim as optim

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

from utils.input_getter import mug_image_getter
import pytorch_lightning as pl
import torch.nn as nn

2025-02-28 17:53:14,531 - root - INFO - AIMET


/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/usr/lib/python3.10/abc.py:106: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)
/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class LightningModel(pl.LightningModule):
    def __init__(self, num_classes=64, learning_rate=2e-5):
        super(LightningModel, self).__init__()
        self.model = models.mobilenet_v3_small(pretrained=True)
        self.model.classifier[3] = nn.Linear(self.model.classifier[3].in_features, num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        images, labels = images.to(self.device), labels.to(self.device)
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)

In [14]:
modelOriginal   = LightningModel.load_from_checkpoint("models/mobilenet_v3-epoch=192-train_loss=0.01.ckpt")
modelOriginal   = modelOriginal.model

import torch.nn as nn
import torchvision.ops as ops
import torchvision.transforms as transforms

class NormalizedV3(torch.nn.Module):

    def __init__(self, mobileNetV3):
        super(NormalizedV3, self).__init__()
        self.model = mobileNetV3
        self.preprocess = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def forward(self, img):
        # Pass the preprocessed image through the model
        return self.model(self.preprocess(img))

model = NormalizedV3(modelOriginal)


/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
totalPost = 0
accuracyPost = 0

from dataset import DatasetReader
from dataset import utils

device = torch.device('cuda')

dataset_coco_val = DatasetReader.COCODataset(annotation_file='../../datasets/coco/annotations/instances_val2017.json',
    image_dir= '../../datasets/coco/val2017',
    target_classes=[s.lower() for s in utils.GLOBAL_CLASSES],
    transform=transforms.Compose([
            transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
            transforms.ToTensor()
        ]),
    min_size = 60)


# batch_size = 256
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=11, prefetch_factor=4, persistent_workers=True)

# Ovo sigurno moze da se optimizuje nekako

model.eval()

for i in range(0, len(dataset_coco_val)):
    totalPost += 1
    image, label, _ = dataset_coco_val[i]
    image = image.unsqueeze(0).to(device)
    label = torch.tensor([label]).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    # print(f"Predicted: {predicted.item()}, Actual: {label.item()}")
    if(i % 1000 == 0):
        print(f"Slika: {i}")
    if predicted.item() == label.item():
        accuracyPost += 1
    

print(f"Accuracy: {accuracyPost/totalPost}")

Total annotations before filtering: 36781
Total annotations after filtering: 8862
Total COCO annotations: 36781
Filtered annotations: 8862
Slika: 0
Slika: 1000
Slika: 2000
Slika: 3000
Slika: 4000
Slika: 5000
Slika: 6000
Slika: 7000
Slika: 8000
Accuracy: 0.791582035657865


In [18]:
import qai_hub

input_shape = (1, 3, 224, 224)

dummy_input = torch.randn(1, 3, 224, 224)
pt_model = torch.jit.trace(model.cpu().eval(), dummy_input)

compile_job = qai_hub.submit_compile_job(
    pt_model,
    name="MobileNetV3FineTuned", # Replace with your model name
    device=qai_hub.Device("Samsung Galaxy S24 (Family)"),
    input_specs=dict(image=input_shape),
)

compile_job.modify_sharing(add_emails=['lowpowervision@gmail.com']) ## Share your model for submission

Uploading tmpdhz4tivm.pt
2025-02-28 18:11:16,508 - root - INFO - Uploading asset to https://tetrahub-qprod-userdata.s3-accelerate.amazonaws.com/models/mngry5keq_A6k1VQQETUVBqjhG.pt?uploadId=5nCHYpprJnFNNsvqPFeuMJ.cdehxMd0Pm3iRju.WGwZ2cDmUSVSCtUTWaQVBQVzO_Z_uappbaeZmVyflq67Ql6xwjpD4ktS8RBvT.Lg0kx0m71CgN..OueNOlnUcx1HMCGRWpllM_.cY7FEEIl9dpYiyx4b7YhohxRT41WsJlak-&partNumber=1&AWSAcces


100%|██████████| 6.51M/6.51M [00:02<00:00, 3.06MB/s]

2025-02-28 18:11:18,739 - root - INFO - Successfully uploaded asset with response status: 200


Scheduled compile job (jpeo82q7g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jpeo82q7g/



In [19]:
profile_job = qai_hub.submit_profile_job(compile_job.get_target_model(), device=qai_hub.Device("Samsung Galaxy S24 (Family)"))

Scheduled profile job (jp12739kg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp12739kg/

